In [1]:
from pathlib import Path

import numpy as np
import xarray as xr
from scipy.interpolate import griddata
from netCDF4 import Dataset

In [2]:
PATH = Path.cwd()/'../Data/NOAA/Atmospherical_Conditions'
PRESSURE_PATH = Path.cwd()/'../Data/NOAA/Pressure_Conditions'
INTERPOLATIONS_PATH = Path.cwd()/'../Data/Interpolations'

IS_SPEEDY_ORDER = True
# IS_SPEEDY_ORDER determines if varibles should follow SPEEDY model variables order. 

if (IS_SPEEDY_ORDER):
    FILES = ['uwnd.2020.nc','vwnd.2020.nc','air.2020.nc','rhum.2020.nc']
else:
    FILES = [k.name for k in PATH.rglob('*.nc')]
    list.sort(FILES)

In [3]:
# NOAA Default pressure leves
PRESSURE_LEVELS_VALUES = [925,850,700,500,300,200,100]

# Grids definition for SPEEDY and NOAA
X_speedy_lon = np.linspace(0,360-3.75,96)
Y_speedy_lat = np.array("-87.159 -83.479 -79.777 -76.070 -72.362 -68.652 -64.942 -61.232 -57.521 -53.810 -50.099 -46.389 -42.678 -38.967 -35.256 -31.545 -27.833 -24.122 -20.411 -16.700 -12.989 -9.278 -5.567 -1.856 1.856 5.567 9.278 12.989 16.700 20.411 24.122 27.833 31.545 35.256 38.967 42.678 46.389 50.099 53.810 57.521 61.232 64.942 68.652 72.362 76.070 79.777 83.479 87.159".split(" "))
Y_speedy_lat = Y_speedy_lat.astype(np.float32)
#Y_speedy_lat = np.flipud(Y_speedy_lat)


## NOAA latitude goes from North To South
X_noaa_lon = np.linspace(0,360-2.5,144)
Y_noaa_lat = np.linspace(90,-90,73)
X_grid_noaa, Y_grid_noaa = np.meshgrid(X_noaa_lon,Y_noaa_lat)


SPEEDY_LON = 96;
SPEEDY_LAT = 48;
SPEEDY_LVL = 7;


NOAA_LON = 144;
NOAA_LAT = 73;

INTERPOLATION_VARIABLES = len(FILES)

# Temporal setting
DATE = '2020-07-01'
TIME = '00:00:00'
## Time is HH:MM:SS in 24-hours format
DATETIME = DATE + 'T' + TIME
FILENAME = DATE.replace('-','') + TIME[:2]

IS_CONVERTION_REQUIRED = True
# IS_CONVERTION_REQUIRED performs Relative humidity convertion to Specific Humidity, if True. 
# If not, Relative Humidity is given

SAVE_AS_GRD = True
# If SAVE_AS_GRD is True, it will convert data into GRD format aditionally to the netCDF files created. 
# The atmospherical variables are in one netCDF file, and the pressure will be on another file.

In [4]:
def read_data(variable,file):
    '''
        input variables
            file          : name of the variable
        output variable
            variable_array: 3-dimensional xarray with all the pressure levels of the given variable
    '''
    variable_path = PATH/file
    variable_array = xr.open_dataset(variable_path)[variable].sel(
        level = PRESSURE_LEVELS_VALUES,
        time = DATETIME)
    return variable_array


def relative2specific(T,RH,p):
    '''
    Parameters
    ----------
        T : Temperature in K.
        RH : Relative humidity in percentage [0,100].
        p : Preassure in mbar.

    Returns
    -------
        specific humidity (dimensionless)

    '''  
    T-=273.15
    p*=100
    RH/=100
    e_s=611.21*np.exp((18.687-T/234.5)*(T/(T+257.14)))
    e=e_s*RH
    w=287.058/461.5*e/(p-e)
    return w/(w+1)

In [5]:
atmospherical_variables = dict()

for file in FILES:
    variable = file.split(".")[0]
    variable_values_by_level = np.zeros((SPEEDY_LVL,NOAA_LAT,NOAA_LON))
    
    variable_array = read_data(variable,file)
    
    for index_pressure_level, pressure in enumerate(PRESSURE_LEVELS_VALUES):
        variable_values_by_level[index_pressure_level,:,:] = variable_array.sel(level=pressure).values
        
    atmospherical_variables[variable] = variable_values_by_level

In [8]:
atmospherical_variables_to_netcdf = dict()
pressure_to_netcdf = dict()

for key, value in atmospherical_variables.items():
    atmospherical_variables_to_netcdf[key] = (("level","lat", "lon"),value)

In [11]:
atmospherical_dataset = xr.Dataset(
    atmospherical_variables_to_netcdf
    ,coords={
        "level": PRESSURE_LEVELS_VALUES,
        "lat": Y_noaa_lat,
        "lon": X_noaa_lon,
    },
    attrs = {
        'long_name': '6-Hourly Sample',
        'Levels': 7,
        'dataset': 'NCEP/DOE AMIP-II Reanalysis (Reanalysis-2)',
        'level_desc': 'Surface',
        'statistic': 'Individual Obs',
    },
)

In [15]:
atmospherical_dataset

<xarray.Dataset>
Dimensions:  (lat: 73, level: 7, lon: 144)
Coordinates:
  * level    (level) int32 925 850 700 500 300 200 100
  * lat      (lat) float64 90.0 87.5 85.0 82.5 80.0 ... -82.5 -85.0 -87.5 -90.0
  * lon      (lon) float64 0.0 2.5 5.0 7.5 10.0 ... 350.0 352.5 355.0 357.5
Data variables:
    uwnd     (level, lat, lon) float64 -2.7 -2.9 -3.1 -3.2 ... -3.0 -3.4 -3.9
    vwnd     (level, lat, lon) float64 -3.7 -3.6 -3.5 -3.3 ... 9.8 9.6 9.5 9.3
    air      (level, lat, lon) float64 274.4 274.4 274.4 ... 192.3 192.3 192.3
    rhum     (level, lat, lon) float64 74.0 74.0 74.0 74.0 ... 78.0 78.0 78.0
Attributes:
    long_name:   6-Hourly Sample
    Levels:      7
    dataset:     NCEP/DOE AMIP-II Reanalysis (Reanalysis-2)
    level_desc:  Surface
    statistic:   Individual Obs

In [17]:
atmospherical_dataset.to_netcdf(INTERPOLATIONS_PATH/("NOAA-" + FILENAME + "-atmospherical_dataset.nc"))

In [23]:
def read_grd(filename):
    ## 1 Forecast, 0 Real
    
    infile = open(filename, "rb")    
    data = np.fromfile(infile , '>f4')
    l = 0
    U = np.empty([nlev,nlat,nlon])
    for k in range(nlev):
        for j in range(nlat):
            for i in range(nlon):
                U[k,j,i] = data[l]
                l=l+1

    V = np.empty([nlev,nlat,nlon])
    for k in range(nlev):
        for j in range(nlat):
            for i in range(nlon):
                V[k,j,i] = data[l]
                l=l+1

    T = np.empty([nlev,nlat,nlon])
    for k in range(nlev):
        for j in range(nlat):
            for i in range(nlon):
                T[k,j,i]  = data[l]
                l=l+1

    SH = np.empty([nlev,nlat,nlon])
    for k in range(nlev):
        for j in range(nlat):
            for i in range(nlon):
                SH[k,j,i] = data[l]
                l=l+1


    P = np.empty([nlat,nlon])
    for j in range(nlat):#range(nlat-1,-1,-1): #range(nlat):
        for i in range(nlon):
            P[j,i] = data[l]
            l=l+1
    return U,V,T,SH,P

In [44]:
FORECASTED_PATH = Path.cwd()/'../Data/SPEEDY'
FORECASTED_FILES = [k.name for k in FORECASTED_PATH.rglob('*.grd')]
nlon = 96;
nlat = 48;
nlev = 7;
[U_SPEEDY,V_SPEEDY,T_SPEEDY,SH_SPEEDY,P_SPEEDY] = read_grd(FORECASTED_PATH/FORECASTED_FILES[2])
variables_SPEEDY = dict()
variables_SPEEDY['uwnd'] = U_SPEEDY
variables_SPEEDY['vwnd'] = V_SPEEDY
variables_SPEEDY['temperature'] = T_SPEEDY
variables_SPEEDY['shum'] = SH_SPEEDY
variables_SPEEDY['pres'] = P_SPEEDY

In [28]:
variables_SPEEDY

{'uwnd': array([[[-15.26973152, -14.32293892, -13.29143238, ..., -17.36129189,
          -16.80972099, -16.10620499],
         [ -1.94902802,  -0.60746342,   0.81258267, ...,  -4.88458061,
           -4.13845348,  -3.14521909],
         [  8.19057274,   8.47264385,   8.55459881, ...,   7.11763   ,
            7.40536356,   7.79868603],
         ...,
         [  0.28492925,   0.54914027,   0.5623548 , ...,  -2.37487483,
           -1.18058264,  -0.28552788],
         [  1.92752063,   1.75443792,   1.4274956 , ...,   1.43438971,
            1.7609334 ,   1.93018436],
         [ -1.47218323,  -2.12309742,  -2.77069736, ...,   0.42628777,
           -0.19146219,  -0.82605428]],
 
        [[-11.10637951,  -9.91398621,  -8.6427021 , ..., -13.93119621,
          -13.13896656, -12.19008446],
         [ -1.46390712,  -0.22849797,   1.14547622, ...,  -3.86665106,
           -3.31811357,  -2.50785804],
         [  5.30531883,   4.49919224,   3.52700329, ...,   7.25284958,
            6.61118746, 

In [45]:
SPEEDY_atmospherical_variables_to_netcdf = dict()
pressure = variables_SPEEDY.pop('pres',None);

for key, value in variables_SPEEDY.items():
    SPEEDY_atmospherical_variables_to_netcdf[key] = (("level","lat", "lon"),value)

In [32]:
SPEEDY_atmospherical_variables_to_netcdf

{'uwnd': (('level', 'lat', 'lon'),
  array([[[-15.26973152, -14.32293892, -13.29143238, ..., -17.36129189,
           -16.80972099, -16.10620499],
          [ -1.94902802,  -0.60746342,   0.81258267, ...,  -4.88458061,
            -4.13845348,  -3.14521909],
          [  8.19057274,   8.47264385,   8.55459881, ...,   7.11763   ,
             7.40536356,   7.79868603],
          ...,
          [  0.28492925,   0.54914027,   0.5623548 , ...,  -2.37487483,
            -1.18058264,  -0.28552788],
          [  1.92752063,   1.75443792,   1.4274956 , ...,   1.43438971,
             1.7609334 ,   1.93018436],
          [ -1.47218323,  -2.12309742,  -2.77069736, ...,   0.42628777,
            -0.19146219,  -0.82605428]],
  
         [[-11.10637951,  -9.91398621,  -8.6427021 , ..., -13.93119621,
           -13.13896656, -12.19008446],
          [ -1.46390712,  -0.22849797,   1.14547622, ...,  -3.86665106,
            -3.31811357,  -2.50785804],
          [  5.30531883,   4.49919224,   3.5270032

In [37]:
speedy_atmospherical_dataset = xr.Dataset(
    SPEEDY_atmospherical_variables_to_netcdf
    ,coords={
        "level": PRESSURE_LEVELS_VALUES,
        "lat": Y_speedy_lat,
        "lon": X_speedy_lon,
    },
    attrs = {
        'long_name': '6-Hourly Sample',
        'Levels': 7,
        'dataset': 'NCEP/DOE AMIP-II Reanalysis (Reanalysis-2)',
        'level_desc': 'Surface',
        'statistic': 'Individual Obs',
    },
)

In [38]:
speedy_atmospherical_dataset

<xarray.Dataset>
Dimensions:      (lat: 48, level: 7, lon: 96)
Coordinates:
  * level        (level) int32 925 850 700 500 300 200 100
  * lat          (lat) float32 -87.159 -83.479 -79.777 ... 79.777 83.479 87.159
  * lon          (lon) float64 0.0 3.75 7.5 11.25 ... 345.0 348.8 352.5 356.2
Data variables:
    uwnd         (level, lat, lon) float64 -15.27 -14.32 -13.29 ... -2.968 -2.71
    vwnd         (level, lat, lon) float64 -5.226 -6.283 -7.153 ... 0.1713 0.282
    temperature  (level, lat, lon) float64 243.3 243.3 243.3 ... 230.1 230.1
    shum         (level, lat, lon) float64 -0.0001799 -0.0001608 ... 8.126e-06
Attributes:
    long_name:   6-Hourly Sample
    Levels:      7
    dataset:     NCEP/DOE AMIP-II Reanalysis (Reanalysis-2)
    level_desc:  Surface
    statistic:   Individual Obs

In [47]:
SPEEDY_pressure_to_netcdf = dict()
SPEEDY_pressure_to_netcdf['pres'] = (("lat", "lon"),pressure)
SPEEDY_pressure_dataset = xr.Dataset(
    SPEEDY_pressure_to_netcdf
    ,coords = {
        "lat": Y_speedy_lat,
        "lon": X_speedy_lon,
    },
    attrs = {
        'long_name': '6-Hourly Pressure at Surface',
        'Levels': 1,
        'units': 'Pascals',
        'precision': -1,
        'GRIB_id': 1,
        'GRIB_name': 'PRES',
        'var_desc': 'Pressure',
        'dataset': 'NCEP/DOE AMIP-II Reanalysis (Reanalysis-2)',
        'level_desc': 'Surface',
        'statistic': 'Individual Obs',
        'parent_stat': 'Other',
        'standard_name': 'pressure',  
    },
)

In [48]:
SPEEDY_pressure_dataset

<xarray.Dataset>
Dimensions:  (lat: 48, lon: 96)
Coordinates:
  * lat      (lat) float32 -87.159 -83.479 -79.777 ... 79.777 83.479 87.159
  * lon      (lon) float64 0.0 3.75 7.5 11.25 15.0 ... 345.0 348.8 352.5 356.2
Data variables:
    pres     (lat, lon) float64 6.929e+04 6.87e+04 ... 1.025e+05 1.025e+05
Attributes:
    long_name:      6-Hourly Pressure at Surface
    Levels:         1
    units:          Pascals
    precision:      -1
    GRIB_id:        1
    GRIB_name:      PRES
    var_desc:       Pressure
    dataset:        NCEP/DOE AMIP-II Reanalysis (Reanalysis-2)
    level_desc:     Surface
    statistic:      Individual Obs
    parent_stat:    Other
    standard_name:  pressure

In [49]:
speedy_atmospherical_dataset.to_netcdf(INTERPOLATIONS_PATH/('SPEEDY-'+FILENAME + "-atmospherical_dataset.nc"))
SPEEDY_pressure_dataset.to_netcdf(INTERPOLATIONS_PATH/('SPEEDY-'+FILENAME + "-pressure_dataset.nc"))